In [28]:
import tensorflow

#from tensorflow.keras.preprocessing import image

AttributeError: ignored

In [ ]:
!pip install tensorflow_datasets

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Input
from tensorflow.keras import callbacks
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import h5py
import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#config = tensorflow.ConfigProto(allow_soft_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#sess=tensorflow.Session(config=config)
import matplotlib.pyplot as plt
import numpy as np
'''

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation="relu",input_shape=(224,224,3)))
model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",metrics = ["accuracy"])
'''

def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)

    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)

    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)

   # x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
   # x = BatchNormalization(name='bn3')(x)
   # x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
   # x = BatchNormalization(name='bn4')(x)
   # x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    # x = MaxPooling2D((2,2), name='pool4')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(128, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(2, activation='softmax', name='fc3')(x)

    model = Model(inputs=input_img, outputs=x)
    return model

model =  build_model()

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

INIT_LR = 1e-4
EPOCHS = 75
BS = 10

train_path = '/content/gdrive/MyDrive/Colab Notebooks/CovidDataset/Train'
val_path = '/content/gdrive/MyDrive/Colab Notebooks/CovidDataset/Val'

training_set = train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=BS, class_mode="categorical")
test_set = test_datagen.flow_from_directory(val_path,target_size=(224,224),batch_size=BS, class_mode="categorical")

opt = Adam(lr=0.0001)
es = callbacks.EarlyStopping(patience=18)
chkpt = callbacks.ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=False)
model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)
history =model.fit_generator(training_set,steps_per_epoch=186//BS,epochs=EPOCHS,validation_data=test_set,validation_steps=94//BS,callbacks=[es,chkpt])


AttributeError: ignored